# Fine-tuning DistilBERT for Customer Support Ticket Classification

In this notebook, we'll fine-tune the DistilBERT model on our cleaned customer support tickets dataset. We'll:
1. Load and preprocess the data
2. Set up DistilBERT model and tokenizer
3. Train for 3-5 epochs
4. Evaluate model performance
5. Save the fine-tuned model

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import evaluate
import os

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
# Load the dataset
df = pd.read_csv('../data/customer_support_tickets_clean_500.csv')
label_mapping = {
    'Billing ': 'billing',
    'Billng': 'billing',
    'billing': 'billing',
    'BILLING': 'billing',
    
    'ACCOUNT': 'account',
    'Accnt': 'account',
    'account': 'account',
    'Account ': 'account',
    
    ' Account ': 'account',  # Add this line
    
    'Other': 'other',
    'other': 'other',
    'Othr': 'other',
    'Other ': 'other',
    'OTHER': 'other',
    
    'TECHNICAL': 'Technical',
    'Tech': 'Technical',
    'technical': 'Technical',
    'Tech-support': 'Technical',
    ' Tech': 'Technical'  # Add this line
}

df['label'] = df['label'].map(label_mapping).fillna(df['label'])

# Encode labels
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Split the data
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label_encoded'])

# Create Dataset objects
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"\nLabel mapping:")
for i, label in enumerate(label_encoder.classes_):
    print(f"{label}: {i}")

Training samples: 377
Validation samples: 95

Label mapping:
Technical: 0
account: 1
billing: 2
other: 3


In [ ]:
# Initialize tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
num_labels = len(label_encoder.classes_)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

# Rename label column to 'labels' for Trainer compatibility
train_dataset = train_dataset.rename_column('label_encoded', 'labels')
val_dataset = val_dataset.rename_column('label_encoded', 'labels')

# Set format for PyTorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
# Define metrics computation
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='../models/distilbert-ticket-classifier',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='../logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
  
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


Evaluation metrics:
eval_loss: 0.9946
eval_accuracy: 0.8947
eval_runtime: 82.7175
eval_samples_per_second: 1.1480
eval_steps_per_second: 0.0730
epoch: 3.0000



In [ ]:
# Train the model
train_results = trainer.train()

# Print training metrics
print("\nTraining metrics:")
print(f"Total training steps: {train_results.global_step}")
print(f"Training loss: {train_results.training_loss:.4f}")

# Evaluate the model
eval_results = trainer.evaluate()
print("\nEvaluation metrics:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")

In [ ]:
# Save the model and tokenizer
output_dir = '../models/distilbert-ticket-classifier'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Save label encoder classes
import json
label_classes = {i: label for i, label in enumerate(label_encoder.classes_)}
with open(os.path.join(output_dir, 'label_mapping.json'), 'w') as f:
    json.dump(label_classes, f)

print(f"\nModel saved to: {output_dir}")
print("Label mapping saved to: label_mapping.json")